# Quantify **Organelle Morphology** - part 2.1
--------------------
Now that all organelles and masks are segmented, we can begin to quantify features of organelle **composition**, **morphology**, **interactions**, and **distribution**. 


## **OBJECTIVE**
### <input type="checkbox"/> Quantify ***organelle composition and morphology***
In this notebook, the logic for quantifying **organelle composition** (how much of each organelle is present) and **morphology** (individual size and shape) is outlined.


--------
## **Organelle Morphology**

### summary of steps

🛠️ **BUILD FUNCTION PROTOTYPE**

- **`0`** - Apply Cell Mask *(preliminary step)*

- **`1`** - Build the list of measurements we want to include from regionprops 

- **`2`** - Add additional measurements as *"extra_properties"* with custom functions.

- **`3`** - Run regionprops and export values as a pandas dataframe

- **`4`** - Add additional measurements
    - surface area
    - surface area to volume ratio

⚙️ **EXECUTE FUNCTION PROTOTYPE**

- Define `_get_org_morphology_3D` function
- Run `_get_org_morphology_3D` function
    - scaled and unscaled
- Compare to finalized `get_org_morphology_3D` function

## **IMPORTS**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block loads all of the necessary python packages and functions you will need for this notebook.

In [ ]:
from pathlib import Path
import os

import napari
from napari.utils.notebook_display import nbscreenshot

from skimage.measure import (regionprops, regionprops_table)

from infer_subc.core.file_io import (read_czi_image,
                                     import_inferred_organelle,
                                     list_image_files)

from infer_subc.core.img import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats import (_assert_uint16_labels)
from infer_subc.utils.stats_helpers import *
from infer_subc.organelles import * 

%load_ext autoreload
%autoreload 2

## **LOAD AND READ IN IMAGE FOR PROCESSING**
> ###### 📝 **Specifically, this will include the raw image and the outputs from segmentation**

#### &#x1F6D1; &#x270D; **User Input Required:**

In [ ]:
## Define the path to the directory that contains the input image folder.
data_root_path = Path(os.getcwd()).parents[1] / "sample_data" /  "example_astrocyte"

## Specify which subfolder that contains the input data and what the file type is. Ex) ".czi" or ".tiff"
in_data_path = data_root_path / "raw"
raw_img_type = ".tiff"

## Specify which subfolder contains the segmentation outputs and their file type
seg_data_path = data_root_path / "seg"
seg_img_type = ".tiff"

## Specify the name of the output folder where quantification results will be saved
out_data_path = data_root_path / "quant"

# Specify which file you'd like to segment from the img_file_list
test_img_n = 0

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

raw_file_list = list_image_files(in_data_path, raw_img_type)
seg_file_list = list_image_files(seg_data_path, seg_img_type)
# pd.set_option('display.max_colwidth', None)
# pd.DataFrame({"Image Name":img_file_list})

In [ ]:
raw_img_name = raw_file_list[test_img_n]

raw_img_data, raw_meta_dict = read_czi_image(raw_img_name)

channel_names = raw_meta_dict['name']
img = raw_meta_dict['metadata']['aicsimage']
scale = raw_meta_dict['scale']
channel_axis = raw_meta_dict['channel_axis']

In [ ]:
## For each import, change the string to match the suffix on the segmentation files (i.e., the stuff following the "-")

# masks

masks_seg_names = ['masks','masks_A', 'masks_B']
for m in masks_seg_names:
    if m in [i.stem.split("-")[-1] for i in seg_file_list]:
        mask_seg = import_inferred_organelle(m, raw_meta_dict, seg_data_path, seg_img_type)
        nuc_seg, cell_seg, cyto_seg = mask_seg
        break

if 'nuc' in [i.stem.split("-")[-1] for i in seg_file_list]:
    nuc_seg = import_inferred_organelle("nuc", raw_meta_dict, seg_data_path, seg_img_type)
    cell_seg = import_inferred_organelle("cell", raw_meta_dict, seg_data_path, seg_img_type)
    cyto_seg = import_inferred_organelle("cyto", raw_meta_dict, seg_data_path, seg_img_type)

#organelles
lyso_seg = import_inferred_organelle("lyso", raw_meta_dict, seg_data_path, seg_img_type)
mito_seg = import_inferred_organelle("mito", raw_meta_dict, seg_data_path, seg_img_type)
golgi_seg = import_inferred_organelle("golgi", raw_meta_dict, seg_data_path, seg_img_type)
perox_seg = import_inferred_organelle("perox", raw_meta_dict, seg_data_path, seg_img_type)
ER_seg = import_inferred_organelle("ER", raw_meta_dict, seg_data_path, seg_img_type)
LD_seg = import_inferred_organelle("LD", raw_meta_dict, seg_data_path, seg_img_type)

-------------------------
## **Visualize with `napari`**

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.add_image(raw_img_data)

In [ ]:
viewer.add_image(cell_seg, colormap='gray', opacity=0.3, blending ='additive')

viewer.add_image(nuc_seg, colormap='blue', blending ='additive')
viewer.add_image(lyso_seg, colormap='cyan', blending ='additive')
viewer.add_image(mito_seg, colormap='green', blending ='additive')
viewer.add_image(golgi_seg, colormap='yellow', blending ='additive')
viewer.add_image(perox_seg, colormap='bop orange', blending ='additive')
viewer.add_image(ER_seg, colormap='red', blending ='additive')
viewer.add_image(LD_seg, colormap='magenta', blending ='additive')

nbscreenshot(viewer, canvas_only=True)

In [ ]:
viewer.close()

-------------------------
# **regionprops**

To measure the amount, size, and shape of the individual organelles, we will utilize `skimage.measure.regionprops`. These measurements can be collected based on pixel/voxel units (assuming the image is isotropic in all dimensions) and or "real-world" units. Since most confocal microscope images are anisotropic (mostly with respect to the Z dimension), we will preferentially utilize real-world units. Luckily, regionprops>=0.20.0 has incorporated a spacing parameter that can handle anisotropic data.

Similar concepts will also be applied to measure the cell, cytoplasm, and nucleus below as well in notebook 1.4.

> ###### **Below, we have listed the properties that are supported in 3D and the properties that support scaling to real-world units**
> ###### 📝 **The regionprop property names correspond to 2D measurements even for those that are able to function in 3D (i.e. "area" is actually "volume" when a 3D image is being processed, etc.)**

In [ ]:
labels = label(nuc_seg )
NUC_CH = 0
rp = regionprops(labels, intensity_image=raw_img_data[NUC_CH], spacing=scale)

supported = [] 
unsupported = []

for prop in rp[0]:
    try:
        rp[0][prop]
        supported.append(prop)
    except NotImplementedError:
        unsupported.append(prop)

print("Supported properties for 3D analysis:")
print("  " + "\n  ".join(supported))

In [ ]:
print("Properties only supported in 2D:")
print("  " + "\n  ".join(unsupported))

###### Below, we have listed the observed properties and the affect of scaling via real-world units (microns):

**Scaled**
>- volume: n * zyx
>- equivalent diameter: n * zyx**(1/3)
>- centroid-0: n * z
>- centroid-1: n * y
>- centroid-2: n * x
>- surface_area: varies by shape and orientation

**Not Scaled**
>- bbox (0-5)
>- euler_number
>- extent
>- max_intensity
>- mean_intensity
>- min_intensity
>- standard_deviation_intensity

# ***BUILD FUNCTION PROTOTYPE***

## **`0` - Apply Cell Mask**
To ensure we are performing single cell analysis, we will apply the cell segmentation as a mask.

> ###### 📝 **The Golgi body segmentation will be the example used throughout this notebook**

In [ ]:
golgi_masked = apply_mask(golgi_seg, cell_seg)

## **`1` - Build the list of measurements we want to include from regionprops**

In [ ]:
# start with LABEL
test_properties = ["label"]

# add position
test_properties = test_properties + ["centroid", "bbox"]

# add area
test_properties = test_properties + ["area", "equivalent_diameter"] # "num_pixels", 

# add shape measurements
test_properties = test_properties + ["extent", "euler_number", "solidity", "axis_major_length"] # "feret_diameter_max", "axis_minor_length"]

# add intensity values (used for quality checks only)
test_properties = test_properties + ["min_intensity", "max_intensity", "mean_intensity"]

test_properties

## **`2` - Add additional measurements as *"extra_properties"* with custom functions**

In [ ]:
# create a function to include the standard deviation of intensities (for quality checks only)
def _standard_deviation_intensity(region, intensities):
    return np.std(intensities[region])

test_extra_properties = [_standard_deviation_intensity]

## **`3` - Run regionprops and export values as a pandas dataframe**

In [ ]:
test_props = regionprops_table(golgi_masked,
                            intensity_image=raw_img_data[0], 
                            properties=test_properties,
                            extra_properties=test_extra_properties,
                            spacing=scale)

In [ ]:
test_props_unscaled = regionprops_table(golgi_masked,
                            intensity_image=raw_img_data[0], 
                            properties=test_properties,
                            extra_properties=test_extra_properties)

In [ ]:
test_props_table = pd.DataFrame(test_props)

test_org_name = "golgi"
test_props_table.insert(1, "object", test_org_name)
test_props_table.rename(columns={"area": "volume"}, inplace=True)

round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
test_props_table.insert(loc=2, column="scale", value=f"{round_scale}")

## **`4` - Add additional measurements**

- surface area
- surface area to volume ratio

In [ ]:
# creating a function to measure the surface area of each object. This function utilizes "marching_cubes" to generate a mesh (non-pixelated object)
def _surface_area_from_props(labels: np.ndarray,
                             props: dict,
                             scale: Union[tuple, None]=None):
    """ 
    a function for getting surface area of volumetric objects

    Parameters:
    ----------
    lables:
        the segmentation np.ndarray with each object labeled a different number
    props:
        region props dictionary resulting from the _my_props_to_dict() function
    spacing:
        tuple of the dimension lengths in the same order as the dimension of your np.ndarray labels input
    """
    surface_areas = np.zeros(len(props["label"]))

    for index, lab in enumerate(props["label"]):
        # this seems less elegant than you might wish, given that regionprops returns a slice,
        # but we need to expand the slice out by one voxel in each direction, or surface area freaks out
        volume = labels[
            max(props["bbox-0"][index] - 1, 0) : min(props["bbox-3"][index] + 1, labels.shape[0]),
            max(props["bbox-1"][index] - 1, 0) : min(props["bbox-4"][index] + 1, labels.shape[1]),
            max(props["bbox-2"][index] - 1, 0) : min(props["bbox-5"][index] + 1, labels.shape[2]),
        ]
        volume = volume == lab
        if scale is None:
            scale=(1.0,) * labels.ndim
        verts, faces, _normals, _values = marching_cubes(volume,
                                                         method="lewiner",
                                                         spacing=scale,
                                                         level=0)
        
        surface_areas[index] = mesh_surface_area(verts, faces)

    return surface_areas

In [ ]:
# props["surface_area"] = surface_area_from_props(nuc_seg, props)
test_surface_area_tab = pd.DataFrame(_surface_area_from_props(golgi_masked, test_props, scale))

test_props_table.insert(12, "surface_area", test_surface_area_tab)
test_props_table.insert(14, "SA_to_volume_ratio", test_props_table["surface_area"].div(test_props_table["volume"]))

pd.set_option('display.max_columns', None)
test_props_table

# ***EXECUTE FUNCTION PROTOTYPE***

## **Define `get_org_morphology_3D()` function**

In [ ]:
def _get_org_morphology_3D(segmentation_img: np.ndarray, 
                           seg_name: str, 
                           intensity_img, 
                           mask: np.ndarray, 
                           scale: Union[tuple, None]=None):
    """
    Parameters
    ------------
    segmentation_img:
        a 3D (ZYX) np.ndarray of segmented objects 
    seg_name: str
        a name or nickname of the object being measured; this will be used for record keeping in the output table
    intensity_img:
        a 3D (ZYX) np.ndarray contain gray scale values from the "raw" image the segmentation is based on )single channel)
    mask:
        a 3D (ZYX) binary np.ndarray mask of the area to measure from
    scale: tuple, optional
        a tuple that contains the real world dimensions for each dimension in the image (Z, Y, X)


    Regionprops measurements:
    ------------------------
    ['label',
    'centroid',
    'bbox',
    'area',
    'equivalent_diameter',
    'extent',
    'feret_diameter_max',
    'euler_number',
    'convex_area',
    'solidity',
    'axis_major_length',
    'axis_minor_length',
    'max_intensity',
    'mean_intensity',
    'min_intensity']

    Additional measurements:
    -----------------------
    ['standard_deviation_intensity',
    'surface_area']


    Returns
    -------------
    pandas dataframe of containing regionprops measurements (columns) for each object in the segmentation image (rows) and the regionprops object
    
    """
    ###################################################
    ## MASK THE ORGANELLE OBJECTS THAT WILL BE MEASURED
    ###################################################
    # in case we sent a boolean mask (e.g. cyto, nucleus, cellmask)
    input_labels = _assert_uint16_labels(segmentation_img)

    # mask
    input_labels = apply_mask(input_labels, mask)

    ##########################################
    ## CREATE LIST OF REGIONPROPS MEASUREMENTS
    ##########################################
    # start with LABEL
    properties = ["label"]

    # add position
    properties = properties + ["centroid", "bbox"]

    # add area
    properties = properties + ["area", "equivalent_diameter"] # "num_pixels", 

    # add shape measurements
    properties = properties + ["extent", "euler_number", "solidity", "axis_major_length"] # ,"feret_diameter_max", "axis_minor_length"]

    # add intensity values (used for quality checks)
    properties = properties + ["min_intensity", "max_intensity", "mean_intensity"]

    #######################
    ## ADD EXTRA PROPERTIES
    #######################
    def standard_deviation_intensity(region, intensities):
        return np.std(intensities[region])

    extra_properties = [standard_deviation_intensity]

    ##################
    ## RUN REGIONPROPS
    ##################
    props = regionprops_table(input_labels, 
                           intensity_image=intensity_img, 
                           properties=properties,
                           extra_properties=extra_properties,
                           spacing=scale)

    props_table = pd.DataFrame(props)
    props_table.insert(0, "object", seg_name)
    props_table.rename(columns={"area": "volume"}, inplace=True)

    if scale is not None:
        round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
        props_table.insert(loc=2, column="scale", value=f"{round_scale}")
    else: 
        props_table.insert(loc=2, column="scale", value=f"{tuple(np.ones(segmentation_img.ndim))}") 

    ##################################################################
    ## RUN SURFACE AREA FUNCTION SEPARATELY AND APPEND THE PROPS_TABLE
    ##################################################################
    surface_area_tab = pd.DataFrame(_surface_area_from_props(input_labels, props, scale))

    props_table.insert(12, "surface_area", surface_area_tab)
    props_table.insert(14, "SA_to_volume_ratio", props_table["surface_area"].div(props_table["volume"]))

    ################################################################
    ## ADD SKELETONIZATION OPTION FOR MEASURING LENGTH AND BRANCHING
    ################################################################


    return props_table

## **Run `_get_org_morphology_3D` function**

In [ ]:
org_img = raw_img_data[0]
seg_name = 'golgi'
org_obj = golgi_seg
cell_mask = cell_seg

# with scale
golgi_table = _get_org_morphology_3D(segmentation_img=org_obj, seg_name=seg_name, intensity_img=org_img, mask=cell_mask, scale=scale)
# without scale
golgi_table_unscaled = _get_org_morphology_3D(segmentation_img=org_obj, seg_name=seg_name, intensity_img=org_img, mask=cell_mask, scale=None)

> **Table with scaled metrics**

In [ ]:
golgi_table

> **Table with unscaled metrics**

In [ ]:
golgi_table_unscaled

In [ ]:
# Will result in FALSE because the _standard_deviation_intensity function was used in the prototyping 
# instead of the standard_deviation_intensity function from stats.py
golgi_table.equals(test_props_table)

## **Compare to finalized `get_org_morphology_3D` function**

In [ ]:
from infer_subc.utils.stats import get_org_morphology_3D

golgi_table_final = get_org_morphology_3D(segmentation_img=org_obj,
                                          seg_name='golgi',
                                          intensity_img=org_img, 
                                          mask=cell_mask, 
                                          scale=scale)

In [ ]:
golgi_table.equals(golgi_table_final)